In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Dataset



In [ ]:
## Load and preprocess data 

from data_pipeline.preprocessing.data_processing import load_and_preprocess_data


image_dir = r'D:\01_Arnaud\Etudes\04_CNAM\RCP209\Projet\DeepLab\data\VOCdevkit\VOC2012\JPEGImages'
mask_dir = r'D:\01_Arnaud\Etudes\04_CNAM\RCP209\Projet\DeepLab\data\VOCdevkit\VOC2012\SegmentationClass'

train_images, val_images, train_masks, val_masks = load_and_preprocess_data(image_dir, mask_dir)


In [ ]:
## Show an image and the corresponding mask, before and after preprocessing

import os
from utils.plotting import display_image_and_mask

image_path = os.path.join(image_dir, os.listdir(image_dir)[1])
mask_path = os.path.join(mask_dir, os.listdir(mask_dir)[0])
display_image_and_mask(image_path, mask_path)

In [ ]:
from utils.plotting import create_class_pixel_table

df = create_class_pixel_table(train_masks)
print(df)

### Effect of data augmentation

In order to reduce overfitting and to increase the number of training exemple, we will proceed with data augmentation. The data augmentation module takes as input the tf.dataset containing the preprocessed image and masks. So it's input shape is ((224,224,3), (224,224,1)).

We will add two types of augmentation:

- random flip and rotation
- random crop: we resize the input at three different scales, 448x448, 672x672 and 896x896 pixels and we random crop a patch with a crop size of 224x224 pixels. This should help the model to capture information at multi-scale.

Next figure displays the augmentation of an image from the dataset

## Backbone architectures

DeepLab algorithm is build upon the Resnet101 architecture trained on imagenet dataset. In order to help building the DeepLab algorithm, we will take a look at its architecture.

In [ ]:
from tensorflow.keras.applications import ResNet101

IMG_SIZE = 224
input_shape=(IMG_SIZE, IMG_SIZE, 3)
base_model = ResNet101(weights='imagenet', include_top=True, input_shape=input_shape)

base_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

base_model.summary()

In [ ]:
from utils.network_inspection import print_all_layers


print_all_layers(base_model)

## Backbone


### Modified Resnet101 architecture



In [17]:
import tensorflow as tf
from tensorflow.keras.models import Model
from deep_lab.layers import ASPP, Backbone, Decoder
from utils.network_inspection import print_trainable_variables


IMG_SIZE = 224
input_shape = (IMG_SIZE, IMG_SIZE, 3)
input = tf.keras.Input(shape=input_shape)

backbone_model = Backbone()
aspp_model = ASPP()
decoder_model = Decoder()

output_backone, low_level_feature = backbone_model(input, training=True)
output_resnet = backbone_model.resnet_model(input, training=True)
output_cascaded = backbone_model.cascaded_blocks(output_resnet, training=True)
output_aspp = aspp_model(output_backone, training=True)
low_level_feature = backbone_model.resnet_model.get_layer('conv2_block3_out').output
output_decoder = decoder_model(output_backone, low_level_feature, training=True)

print("Backbone model:")
backbone_model = Model(inputs=input, outputs=output_backone)
backbone_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
backbone_model.summary()
print_trainable_variables(backbone_model)
# tf.keras.utils.plot_model(backbone_model, show_shapes=True, dpi=48)

print("ResNet without cascaded blocks model:")
resnet_model = Model(inputs=input, outputs=output_resnet)
resnet_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
resnet_model.summary()
print_trainable_variables(resnet_model)
# tf.keras.utils.plot_model(resnet_model, show_shapes=True, dpi=48)

print("Cascaded blocks model:")
cascaded_model = Model(inputs=output_resnet, outputs=output_cascaded)
cascaded_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
cascaded_model.summary()
print_trainable_variables(cascaded_model)
tf.keras.utils.plot_model(cascaded_model, show_shapes=True, dpi=48)

print("ASPP model:")
aspp_model = Model(inputs=output_backone, outputs=output_aspp)
aspp_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
aspp_model.summary()
print_trainable_variables(aspp_model)
tf.keras.utils.plot_model(aspp_model, show_shapes=True, dpi=48)

print("Decodeur model")
decoder_model = Model(inputs=output_backone, outputs=output_decoder)
decoder_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
decoder_model.summary()
print_trainable_variables(decoder_model)
tf.keras.utils.plot_model(decoder_model, show_shapes=True, dpi=48)

Backbone model:


Model: "functional_22"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_26 (InputLayer)     │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ backbone (Backbone)             │ [(None, 14, 14, 2048), │    56,064,384 │
│                                 │ (None, 56, 56, 256)]   │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 56,064,384 (213.87 MB)

 Trainable params: 55,940,608 (213.40 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_27      │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_27[1… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[1][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[1][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[1][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[1][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[1][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[1][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[1][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 5,668,693 (21.62 MB)

 Trainable params: 5,664,629 (21.61 MB)

 Non-trainable params: 4,064 (15.88 KB)

Layer: input_layer_27, Weights: []
Non trainable variables: []
Layer: conv1_pad, Weights: []
Non trainable variables: []
Layer: conv1_conv, Weights: [<KerasVariable shape=(7, 7, 3, 64), dtype=float32, path=conv1_conv/kernel>, <KerasVariable shape=(64,), dtype=float32, path=conv1_conv/bias>]
Non trainable variables: []
Layer: conv1_bn, Weights: [<KerasVariable shape=(64,), dtype=float32, path=conv1_bn/gamma>, <KerasVariable shape=(64,), dtype=float32, path=conv1_bn/beta>]
Non trainable variables: [<KerasVariable shape=(64,), dtype=float32, path=conv1_bn/moving_mean>, <KerasVariable shape=(64,), dtype=float32, path=conv1_bn/moving_variance>]
Layer: conv1_relu, Weights: []
Non trainable variables: []
Layer: pool1_pad, Weights: []
Non trainable variables: []
Layer: pool1_pool, Weights: []
Non trainable variables: []
Layer: conv2_block1_1_conv, Weights: [<KerasVariable shape=(1, 1, 64, 64), dtype=float32, path=conv2_block1_1_conv/kernel>, <KerasVariable shape=(64,), dtype=float32, path=conv

In [24]:
from utils.network_inspection import print_trainable_variables

IMG_SIZE = 224
input_shape = (IMG_SIZE, IMG_SIZE, 3)
input = tf.keras.Input(shape=input_shape)

backbone_model = Backbone()
aspp_model = ASPP()
decoder_model = Decoder()

backbone_model.build(input_shape) 
print_trainable_variables(model=backbone_model)
# tf.keras.utils.plot_model(backbone_model, show_shapes=True, dpi=48)

aspp_model = ASPP()
decoder_model = Decoder()

d:\01_Arnaud\Etudes\04_CNAM\RCP209\Projet\DeepLab\.venv\Lib\site-packages\keras\src\layers\layer.py:372: UserWarning: `build()` was called on layer 'backbone', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
